In [0]:
%matplotlib inline
!pip install category_encoders
from google.colab import drive
import os
import time
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/AML")
pd.set_option('display.max_rows', 20)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential

Using TensorFlow backend.


In [0]:
data=load_digits()

In [0]:
data.images.shape

(1797, 8, 8)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data.images, data.target, random_state=1)

In [0]:
## flatenning
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])

In [0]:
X_train=X_train.astype('float64')

In [0]:
X_train=X_train/255

In [0]:
X_test=X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
X_test=X_test.astype('float64')
X_test=X_test/255

In [0]:
cls=len(np.unique(data.target))

In [0]:
y_test_n=to_categorical(y_test, cls)
y_train_n=to_categorical(y_train, cls)

In [0]:
def build_model(neurons=32, l2_val=0.1):
  inp=X_train[0].shape
  model=Sequential([Dense(neurons,activation='relu',kernel_regularizer=l2(l=l2_val), input_shape=inp),Dense(neurons,activation='relu',kernel_regularizer=l2(l=l2_val)), Dense(cls,activation='softmax',kernel_regularizer=l2(l=l2_val)) ])
  model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
  return model
  

In [0]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=1)

In [0]:
model_mlp=KerasClassifier(build_model)
param_grid={'neurons':[32,64,128], 'l2_val':[0.001, 0.03,1,3,10], 'epochs':[30,]}
grid_mlp=GridSearchCV(model_mlp,param_grid=param_grid, return_train_score=True, cv = sss)


In [0]:
grid_mlp.fit(X_train, y_train_n)

Epoch 1/30
1010/1010 [==============================] - 0s 244us/step - loss: 2.3800 - accuracy: 0.1307
Epoch 2/30
1010/1010 [==============================] - 0s 112us/step - loss: 2.3524 - accuracy: 0.2693
Epoch 3/30
1010/1010 [==============================] - 0s 134us/step - loss: 2.3265 - accuracy: 0.3505
Epoch 4/30
1010/1010 [==============================] - 0s 118us/step - loss: 2.2969 - accuracy: 0.3762
Epoch 5/30
1010/1010 [==============================] - 0s 115us/step - loss: 2.2551 - accuracy: 0.4347
Epoch 6/30
1010/1010 [==============================] - 0s 120us/step - loss: 2.1911 - accuracy: 0.4465
Epoch 7/30
1010/1010 [==============================] - 0s 118us/step - loss: 2.0951 - accuracy: 0.4861
Epoch 8/30
1010/1010 [==============================] - 0s 117us/step - loss: 1.9743 - accuracy: 0.4792
Epoch 9/30
1010/1010 [==============================] - 0s 128us/step - loss: 1.8438 - accuracy: 0.5178
Epoch 10/30
1010/1010 [==============================] - 0s 124u

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=1, random_state=1, test_size=0.25,
            train_size=None),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f024f0f1438>,
             iid='deprecated', n_jobs=None,
             param_grid={'epochs': [30], 'l2_val': [0.001, 0.03, 1, 3, 10],
                         'neurons': [32, 64, 128]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [0]:
grid_stat=pd.DataFrame(grid_mlp.cv_results_)
grid_stat.pivot_table(index=["param_neurons", "param_l2_val"],values=["mean_test_score", "mean_train_score"] )

mean_test_score  mean_train_score
param_neurons param_l2_val                                   
32            0.001                0.860534          0.861386
              0.030                0.106825          0.105941
              1.000                0.106825          0.105941
              3.000                0.106825          0.105941
              10.000               0.106825          0.105941
64            0.001                0.913947          0.924752
              0.030                0.106825          0.105941
              1.000                0.106825          0.105941
              3.000                0.106825          0.105941
              10.000               0.106825          0.105941
128           0.001                0.940653          0.949505
              0.030                0.106825          0.105941
              1.000                0.106825          0.105941
              3.000                0.106825          0.105941
              10.000               0.106825          0.105941

In [0]:
grid_mlp.score(X_test, y_test_n)

450/450 [==============================] - 0s 169us/step


0.9288889169692993

In [0]:
print(grid_mlp.best_params_)

{'epochs': 30, 'l2_val': 0.001, 'neurons': 128}


Our neural net is giving good score on the test based on params picked by grid search.